In [2]:
! pip install simpy

In [0]:
import simpy
import random
import math
import numpy as np

In [0]:
RANDOM_SEED = 42
random_seeds = [9, 8, 7, 6, 5, 4, 3, 2, 1, 10]
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
SERVICE_RANGE= [1,7]        # service time of op2 is uniformly distributed between 1 and 7
INTERARRIVAL_RATE = 1/6     # given that 1/6 customer per minute
NUM_MACHINES = 100          # Number of machines in the answering system
RECOGNITION_RATE = 1/5      # given that 1/5 service per minute
PATIENCE = 0                # patiance for firstcall
PATIENCE2 = 10              # patiance for operator queues
drop_customers = []         # their phones never opened...
unsatisfied_customers_10 = []  # customers who waited 10 mins  
unsatisfied_customers = []  # customers who  misdirected 
queue_w_times_op1 = []      # queue waiting times for operator1
queue_w_times_op2 = []      # queue waiting times for operator2
service_times_op1 = []      # (with operator 1)
service_times_op2 = []      # (with operator 2)
SIMULATION_TIME=0           # it the end of the simulation, it will be filled
answering_machine_times = []   # duration of time spent during voice recognition
system_time = []            # total number of minutes a served customer spent in the system
twt_tst_ratio = []          # total waiting time to total system time ratio
queue_length_time_op1 = [] # Queue length info for operator 1
queue_length_time_op2 = [] # Queue length info for operator 1
#to calculate lognormal distribution, we need mu and sigma. 
mean = 12
std = 6

MU = math.log(mean**2/math.sqrt(mean**2 + std**2))

VARIANCE = math.log((mean**2 + std**2) / mean**2 )

SIGMA = math.sqrt(VARIANCE)




In [0]:
# This function generates the breaks using poisson dist.
# Generates breaks for every 8*60 mins.
def generate_breaks(env):
    current_shift = -1
    while(1):
        global simulation_finished
        global op1_breaktime
        global op2_breaktime
        if(simulation_finished):
            break
        if(current_shift != env.now //(8*60)):
            current_shift = env.now //(8*60)
            op1_breaktime = sorted([(current_shift*8*60) + random.uniform(0,8*60) for i in range(np.random.poisson(8))])
            op2_breaktime = sorted([(current_shift*8*60) + random.uniform(0,8*60) for i in range(np.random.poisson(8))])
            yield env.timeout(8*60)

In [0]:
# While taking breaks operator himself sends a request to himself.
# When he got the service right, checks if the queue is empty. If so, takes break, else makes another request. 
def take_break_1(env):
    while(1):
        global simulation_finished
        global op1_breaktime
        global op2_breaktime
        if(simulation_finished):
            break
        if(len(op1_breaktime) ==0): # if all breaks are taken
            yield env.timeout((env.now//(8*60) + 1)*(8*60) - env.now)
            if(simulation_finished):
                break
        if(len(op1_breaktime) > 0 and env.now >= op1_breaktime[0]): 
            with operator1.request() as req: # make a request to himself
                yield req
                #print("Operator 1 tries to take a break at %s" % str(env.now))
                if(len(operator1.queue) == 0): # if queue is empty then take a break.
                    del op1_breaktime[0]
                    yield env.timeout(3)
                    #print("Operator 1 comes again at %s" % str(env.now))
        else:
            if(len(op1_breaktime) ==0): 
                yield env.timeout((env.now//(8*60) + 1)*(8*60) - env.now)
                if(simulation_finished):
                    break
            else:
                yield env.timeout(op1_breaktime[0] - env.now)

# The same as above but for op2.
def take_break_2(env):
    while(1):
        global simulation_finished
        global op1_breaktime
        global op2_breaktime
        if(simulation_finished):
            break
        if(len(op2_breaktime) == 0):
            yield env.timeout((env.now//(8*60) + 1)*(8*60) - env.now)
            if(simulation_finished):
                break
        if(len(op2_breaktime)>0 and env.now >= op2_breaktime[0]):
            with operator2.request() as req:
                yield req
                #print("Operator 2 tries to take a break at %s" % str(env.now))
                if(len(operator2.queue) == 0):
                    del op2_breaktime[0]
                    yield env.timeout(3)
                    #print("Operator 2 comes again at %s" % str(env.now))
        else:
            if(len(op2_breaktime) ==0):
                yield env.timeout((env.now//(8*60) + 1)*(8*60) - env.now)
                if(simulation_finished):
                    break
            else:
                yield env.timeout(op2_breaktime[0] - env.now)

In [0]:
def calculateAvgQueueLength(queue_info):
    avg = 0
    for i in range(len(queue_info)-1):
        avg += (queue_info[i+1][0] - queue_info[i][0])* queue_info[i][1]
    return avg/SIMULATION_TIME

In [0]:
class AnswerMachine(object):
    
    def __init__(self, env, num_machines, recognize_rate):
        self.env = env
        self.machine = simpy.Resource(env, num_machines) #voice recognition resources
        self.recognize_rate =recognize_rate

    # voice recognition
    def recognize(self, customer):
        duration = random.expovariate(self.recognize_rate) 
        yield self.env.timeout(duration)
       

class Customer(object):
    def __init__(self, name, env,answeringMachine):
        self.env = env
        self.name = name
        self.arrival_t = self.env.now
        self.redirection_time = 0
        self.leave_time = 0
        self.action = env.process(self.firstCall(answeringMachine))
        self.total_waiting_time=0
        self.total_service_time=0


    # Getting service from an operator.
    def ask_question(self,operator_num):
        if operator_num == 2:
           duration = random.uniform(*SERVICE_RANGE)
           yield self.env.timeout(duration)
           service_times_op2.append(duration)
           self.total_service_time+=duration
           self.leave_time = self.env.now

        else:
           duration = random.lognormvariate(MU, SIGMA)
           yield self.env.timeout(duration)
           service_times_op1.append(duration)
           self.total_service_time+=duration
           self.leave_time = self.env.now

    # routing from answering system to operators.
    def redirect(self, operator, operator_num):
        self.redirection_time =self.env.now
        if(operator_num == 1):
            queue_length_time_op1.append((self.redirection_time, len(operator1.queue)+1))
        else:
            queue_length_time_op2.append((self.redirection_time, len(operator2.queue)+1))
        with operator.request() as req:
            results = yield req | env.timeout(PATIENCE2)
            wait_in_op_queue = self.env.now - self.redirection_time
            self.total_waiting_time=wait_in_op_queue
            if operator_num ==2:
                queue_w_times_op2.append(wait_in_op_queue)
            else:
                queue_w_times_op1.append(wait_in_op_queue)
            if req in results:
              if(operator_num == 1):
                  queue_length_time_op1.append((self.env.now, len(operator1.queue)))
              else:
                  queue_length_time_op2.append((self.env.now, len(operator2.queue)))
              # We got to the counter
              #print('%s is assigned to an operator at %g' % (self.name, self.env.now))
              yield self.env.process(self.ask_question(operator_num))
              #print('%s is done at %g' % (self.name, self.env.now))
              global last_customer_time
              last_customer_time = self.env.now

            else:
              # We reneged because waiting time exceeded 10 mins
              #print('%7.4f %s: RENEGED ' % (self.env.now, self.name))
              unsatisfied_customers_10.append(self.name)
              self.leave_time = self.env.now
              if(operator_num == 1):
                  queue_length_time_op1.append((self.env.now, len(operator1.queue)))
              else:
                  queue_length_time_op2.append((self.env.now, len(operator2.queue)))

    # route customer to either true operator or misroute
    def probabilty(self):
        result = random.uniform(0,100)
        if result <= 10:
          # caller is misrouted
          #print("%s MISDIRECTED :(  at %g " %(self.name, self.env.now))
          unsatisfied_customers.append(self.name)
        elif result <=37:
          # caller is assigned to op1
          #print("%s redirected to operator 1 at %g " %(self.name, self.env.now))
          yield self.env.process(self.redirect(operator1, 1))
        else:
          # caller is assigned to op2
          #print("%s redirected to operator 2 at %g " %(self.name, self.env.now))
          yield self.env.process(self.redirect(operator2, 2))

    # calling answering system
    def firstCall(self,answeringMachine):
      #print('%s calls the service at %g .' % (self.name, self.env.now))
      with answeringMachine.machine.request() as request:
          results = yield request | env.timeout(PATIENCE)
          if request in results:
              # We answered by the machine
              #print('%s is answered by an answerMachine at %g' % (self.name, self.env.now))
              yield env.process(answeringMachine.recognize(self.name))
              aw_time = self.env.now - self.arrival_t
              self.total_service_time+=aw_time
              answering_machine_times.append(aw_time)
              #print('%s leaves the answering machine to be directed at %g it took %g. minutes' 
              #      % (self.name, self.env.now, aw_time))

              yield self.env.process(self.probabilty())
              twt_tst_ratio.append((self.total_waiting_time/(self.total_waiting_time+self.total_service_time), self.name, self.arrival_t, self.redirection_time, self.leave_time))
              
              #print('%s left the system at %g' % (self.name, self.env.now))
              system_time.append(self.env.now - self.arrival_t)

          else:
              # We dropped
              #print('%7.4f %s: DROPPED because there is no available ANSWERING MACHINE' % (self.env.now, self.name))
              drop_customers.append(self.name)


In [0]:
def setup(env, num_machines, recognize_rate, inter_rate, num_customers=1000):
   # Create the aw
   AnswerMachine1 = AnswerMachine(env, num_machines, recognize_rate)
   for i in range(num_customers):
        yield env.timeout(random.expovariate(inter_rate))
        customer = Customer('Cust %s' %(i+1), env, AnswerMachine1) 
   global simulation_finished
   simulation_finished = True
    
def simulation_results(seed):
    return {"Seed": seed,
            "Answer_System_Util": (sum(answering_machine_times)/NUM_MACHINES/SIMULATION_TIME)*100,
            "Operator1_Util": (sum(service_times_op1)/SIMULATION_TIME)*100,
            "Operator2_Util": (sum(service_times_op2)/SIMULATION_TIME)*100,
            "Avg_queue_len_1": calculateAvgQueueLength(queue_length_time_op1),
            "Avg_queue_len_2": calculateAvgQueueLength(queue_length_time_op2),
            "Max_total_wait_time": sum(queue_w_times_op1) + sum(queue_w_times_op2),
            "Max_total_system_time": sum(system_time),
            "MTWT/MTST": ((sum(queue_w_times_op1) + sum(queue_w_times_op2)) / sum(system_time))*100,
            "avg_wait_time" : (sum(queue_w_times_op1)+sum(queue_w_times_op2)) /(len(queue_w_times_op1)+len(queue_w_times_op2)),
            "Num_Of_Unsatisfied_Customers": len(unsatisfied_customers)+len(unsatisfied_customers_10),
            "Simulation_time": SIMULATION_TIME
           }

In [0]:
results_1000 = []
results_5000 = []
op1_breaktime = []
op2_breaktime = []

**Running Simulation**

You may want to ignore the following two cell's outputs. Print statements are used for double check.



In [0]:
for i in range(10):
    # Initializing data holders and parameters
    last_customer_time = 0
    simulation_finished = False
    RANDOM_SEED = random_seeds[i]
    random.seed(RANDOM_SEED)
    np.random.seed(RANDOM_SEED)
    drop_customers = []         # their phones never opened...
    unsatisfied_customers_10 = []  # customers who waited 10 mins  
    unsatisfied_customers = []  # customers who  misdirected 
    queue_w_times_op1 = []      # queue waiting times for operator1
    queue_w_times_op2 = []      # queue waiting times for operator2
    service_times_op1 = []      # (with operator 1)
    service_times_op2 = []      # (with operator 2)
    SIMULATION_TIME=0           # it the end of the simulation, it will be filled
    answering_machine_times = []   # duration of time spent during voice recognition
    system_time = []            # total number of minutes a served customer spent in the system
    twt_tst_ratio = []          # total waiting time to total system time ratio
    queue_length_time_op1 = [] # Queue length info for operator 1
    queue_length_time_op2 = [] # Queue length info for operator 1
    env = simpy.Environment()
    operator1 = simpy.Resource(env, capacity = 1) #no breaks added yet
    operator2 = simpy.Resource(env, capacity = 1)
    env.process(setup(env, NUM_MACHINES, RECOGNITION_RATE, INTERARRIVAL_RATE,1000))
    env.process(generate_breaks(env))
    env.process(take_break_1(env))
    env.process(take_break_2(env))
    env.run()
    SIMULATION_TIME=last_customer_time
    results_1000.append(simulation_results(RANDOM_SEED))

In [0]:
for i in range(10):
    # Initializing data holders and parameters
    last_customer_time = 0
    simulation_finished = False
    RANDOM_SEED = random_seeds[i]
    random.seed(RANDOM_SEED)
    np.random.seed(RANDOM_SEED)
    drop_customers = []         # their phones never opened...
    unsatisfied_customers_10 = []  # customers who waited 10 mins  
    unsatisfied_customers = []  # customers who  misdirected 
    queue_w_times_op1 = []      # queue waiting times for operator1
    queue_w_times_op2 = []      # queue waiting times for operator2
    service_times_op1 = []      # (with operator 1)
    service_times_op2 = []      # (with operator 2)
    SIMULATION_TIME=0           # it the end of the simulation, it will be filled
    answering_machine_times = []   # duration of time spent during voice recognition
    system_time = []            # total number of minutes a served customer spent in the system
    twt_tst_ratio = []          # total waiting time to total system time ratio
    queue_length_time_op1 = [] # Queue length info for operator 1
    queue_length_time_op2 = [] # Queue length info for operator 1
    #Initializing simulation
    env = simpy.Environment()
    operator1 = simpy.Resource(env, capacity = 1) #no breaks added yet
    operator2 = simpy.Resource(env, capacity = 1)
    env.process(setup(env, NUM_MACHINES, RECOGNITION_RATE, INTERARRIVAL_RATE,5000))
    env.process(generate_breaks(env))
    env.process(take_break_1(env))
    env.process(take_break_2(env))
    env.run()
    SIMULATION_TIME=last_customer_time
    #Results
    results_5000.append(simulation_results(RANDOM_SEED))

In [0]:
import pandas as pd

**Statistics for 1000 Customers**

In [14]:
data1k = pd.DataFrame.from_dict(results_1000)
data1k

,Seed,Answer_System_Util,Operator1_Util,Operator2_Util,Avg_queue_len_1,Avg_queue_len_2,Max_total_wait_time,Max_total_system_time,MTWT/MTST,avg_wait_time,Num_Of_Unsatisfied_Customers,Simulation_time
0,9,0.788641,44.766676,41.131224,0.365499,0.199143,1780.102830,11830.760778,15.046394,2.000116,162,6100.106421
1,8,0.864075,48.845112,42.202846,0.375994,0.279054,2107.360894,12601.589548,16.722977,2.308172,138,5913.724598
2,7,0.892891,48.707365,44.304568,0.430205,0.277755,2168.933119,12215.628641,17.755395,2.412606,167,5511.047383
3,6,0.855937,44.515552,42.466448,0.357070,0.227717,1873.927049,12250.045612,15.297307,2.061526,143,6012.501178
4,5,0.764848,44.452163,42.188343,0.393729,0.258601,1925.787356,11888.388999,16.198893,2.123250,156,6107.328698
5,4,0.838588,41.950022,40.675488,0.383868,0.203067,1647.614339,11682.553645,14.103204,1.874419,164,6027.557166
6,3,0.807514,45.809346,41.386881,0.311904,0.222411,1918.974472,12020.552104,15.964113,2.090386,142,6014.720182
7,2,0.843235,42.722403,42.787651,0.412428,0.261767,1917.963580,12043.843329,15.924847,2.112295,157,5962.236896
8,1,0.786278,43.140918,41.660686,0.364361,0.221695,1871.003606,11752.727701,15.919739,2.067407,151,6046.479640
9,10,0.812896,46.978923,40.368514,0.389268,0.255671,1984.780962,12092.882657,16.412803,2.188292,150,5993.997495


In [35]:
data1k.drop(columns= ["Seed"]).describe(include='all')

,Answer_System_Util,Operator1_Util,Operator2_Util,Avg_queue_len_1,Avg_queue_len_2,Max_total_wait_time,Max_total_system_time,MTWT/MTST,avg_wait_time,Num_Of_Unsatisfied_Customers,Simulation_time
count,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
mean,0.825490,45.188848,41.917265,0.378433,0.240688,1919.644821,12037.897301,15.934567,2.123847,153.000000,5968.969966
std,0.040138,2.386984,1.145962,0.032439,0.029515,148.919940,272.957663,0.988018,0.151788,9.899495,171.005205
min,0.764848,41.950022,40.368514,0.311904,0.199143,1647.614339,11682.553645,14.103204,1.874419,138.000000,5511.047383
25%,0.793359,43.468729,41.195139,0.364645,0.221874,1871.734467,11845.167833,15.452915,2.062996,144.750000,5970.177046
50%,0.825742,44.641114,41.924514,0.379931,0.241694,1918.469026,12032.197716,15.944480,2.101340,153.500000,6013.610680
75%,0.852762,46.686528,42.400547,0.392614,0.260975,1970.032561,12184.942145,16.359325,2.172031,160.750000,6041.749021
max,0.892891,48.845112,44.304568,0.430205,0.279054,2168.933119,12601.589548,17.755395,2.412606,167.000000,6107.328698


In [37]:
data1k["Num_Of_Unsatisfied_Customers"].describe(include='all')

count     10.000000
mean     153.000000
std        9.899495
min      138.000000
25%      144.750000
50%      153.500000
75%      160.750000
max      167.000000
Name: Num_Of_Unsatisfied_Customers, dtype: float64

**Statistics for 5000 Customers**

In [16]:
data5k = pd.DataFrame.from_dict(results_5000)
data5k

,Seed,Answer_System_Util,Operator1_Util,Operator2_Util,Avg_queue_len_1,Avg_queue_len_2,Max_total_wait_time,Max_total_system_time,MTWT/MTST,avg_wait_time,Num_Of_Unsatisfied_Customers,Simulation_time
0,9,0.816389,45.295703,40.146492,0.375693,0.228977,9544.663234,60554.905307,15.761998,2.115395,774,30530.235427
1,8,0.863237,45.530369,41.182325,0.421645,0.263155,10263.098073,61202.092178,16.769195,2.279169,811,29438.304469
2,7,0.865630,46.316251,42.444939,0.362172,0.258059,9949.399575,60835.563701,16.354578,2.204120,779,29024.045583
3,6,0.846834,45.000056,41.086000,0.387272,0.232007,9446.518270,60940.434309,15.501232,2.094107,775,30154.056226
4,5,0.794023,43.829267,41.544748,0.346484,0.251297,9122.199506,59894.263477,15.230506,2.010624,717,30812.712688
5,4,0.829542,44.940208,40.584073,0.374683,0.229048,9279.651433,60474.783548,15.344663,2.070426,781,30386.747064
6,3,0.801684,43.836094,40.446680,0.363539,0.232733,9454.318219,59665.136921,15.845632,2.096301,787,30532.362053
7,2,0.832389,46.019526,41.080493,0.402647,0.242375,9849.632142,61013.186382,16.143448,2.173352,790,30036.334473
8,1,0.807802,43.083977,41.617893,0.359713,0.213068,8821.645999,58782.671088,15.007222,1.964294,758,30191.207789
9,10,0.815339,44.747231,41.372703,0.363415,0.249846,9682.323718,59927.059047,16.156848,2.139740,780,29969.330768


In [36]:
data5k.drop(columns= ["Seed"]).describe(include='all')

,Answer_System_Util,Operator1_Util,Operator2_Util,Avg_queue_len_1,Avg_queue_len_2,Max_total_wait_time,Max_total_system_time,MTWT/MTST,avg_wait_time,Num_Of_Unsatisfied_Customers,Simulation_time
count,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
mean,0.827287,44.859868,41.150635,0.375726,0.240057,9541.345017,60329.009596,15.811532,2.114753,775.200000,30107.533654
std,0.024898,1.022427,0.661072,0.022516,0.015549,419.134279,753.919047,0.552148,0.091463,24.484916,536.401874
min,0.794023,43.083977,40.146492,0.346484,0.213068,8821.645999,58782.671088,15.007222,1.964294,717.000000,29024.045583
25%,0.809686,44.063878,40.708178,0.362483,0.229788,9321.368142,59902.462369,15.383805,2.076347,774.250000,29986.081694
50%,0.822965,44.970132,41.134163,0.369111,0.237554,9499.490726,60514.844427,15.803815,2.105848,779.500000,30172.632008
75%,0.843223,45.471703,41.501737,0.384377,0.250934,9807.805036,60914.216657,16.153498,2.164949,785.500000,30494.363336
max,0.865630,46.316251,42.444939,0.421645,0.263155,10263.098073,61202.092178,16.769195,2.279169,811.000000,30812.712688


In [38]:
data5k["Num_Of_Unsatisfied_Customers"].describe(include='all')

count     10.000000
mean     775.200000
std       24.484916
min      717.000000
25%      774.250000
50%      779.500000
75%      785.500000
max      811.000000
Name: Num_Of_Unsatisfied_Customers, dtype: float64

**To see the graphics remove the comments and run that cell.** <br>
**Do not forget to install _plotly_ and import it.** <br>
All the graphs generated using the cells below are added to the report.

In [17]:
"""!pip install plotly
import plotly.graph_objects as go"""

'!pip install plotly\nimport plotly.graph_objects as go'

In [18]:
"""
fig = go.Figure([go.Bar(x = random_seeds , y = data1k["Answer_System_Util"] )])
fig.update_layout(
    title="Answering System Utilization for 1000 Customers",
    xaxis_title="Random Seeds",
    yaxis_title="Utilization Percentage (%)",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"
    )
)
fig.show()
"""

'\nfig = go.Figure([go.Bar(x = random_seeds , y = data1k["Answer_System_Util"] )])\nfig.update_layout(\n    title="Answering System Utilization for 1000 Customers",\n    xaxis_title="Random Seeds",\n    yaxis_title="Utilization Percentage (%)",\n    font=dict(\n        family="Courier New, monospace",\n        size=18,\n        color="#7f7f7f"\n    )\n)\nfig.show()\n'

In [19]:
"""fig = go.Figure([go.Bar(x = random_seeds , y = data5k["Answer_System_Util"] )])
fig.update_layout(
    title="Answering System Utilization for 5000 Customers",
    xaxis_title="Random Seeds",
    yaxis_title="Utilization Percentage (%)",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"
    )
)
fig.show()
"""

'fig = go.Figure([go.Bar(x = random_seeds , y = data5k["Answer_System_Util"] )])\nfig.update_layout(\n    title="Answering System Utilization for 5000 Customers",\n    xaxis_title="Random Seeds",\n    yaxis_title="Utilization Percentage (%)",\n    font=dict(\n        family="Courier New, monospace",\n        size=18,\n        color="#7f7f7f"\n    )\n)\nfig.show()\n'

In [20]:
"""fig = go.Figure([go.Bar(x = random_seeds , y = data1k["Operator1_Util"] )])
fig.update_layout(
    title="Utilization of Operator 1 for 1000 Customers",
    xaxis_title="Random Seeds",
    yaxis_title="Utilization Percentage",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"
    )
)
fig.show()"""

'fig = go.Figure([go.Bar(x = random_seeds , y = data1k["Operator1_Util"] )])\nfig.update_layout(\n    title="Utilization of Operator 1 for 1000 Customers",\n    xaxis_title="Random Seeds",\n    yaxis_title="Utilization Percentage",\n    font=dict(\n        family="Courier New, monospace",\n        size=18,\n        color="#7f7f7f"\n    )\n)\nfig.show()'

In [21]:
"""fig = go.Figure([go.Bar(x = random_seeds , y = data5k["Operator1_Util"] )])
fig.update_layout(
    title="Utilization of Operator 1 for 5000 Customers",
    xaxis_title="Random Seeds",
    yaxis_title="Utilization Percentage",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"
    )
)
fig.show()
"""

'fig = go.Figure([go.Bar(x = random_seeds , y = data5k["Operator1_Util"] )])\nfig.update_layout(\n    title="Utilization of Operator 1 for 5000 Customers",\n    xaxis_title="Random Seeds",\n    yaxis_title="Utilization Percentage",\n    font=dict(\n        family="Courier New, monospace",\n        size=18,\n        color="#7f7f7f"\n    )\n)\nfig.show()\n'

In [22]:
"""fig = go.Figure([go.Bar(x = random_seeds , y = data1k["Operator2_Util"] )])
fig.update_layout(
    title="Utilization of Operator 2 for 1000 Customers",
    xaxis_title="Random Seeds",
    yaxis_title="Utilization Percentage",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"
    )
)
fig.show()"""

'fig = go.Figure([go.Bar(x = random_seeds , y = data1k["Operator2_Util"] )])\nfig.update_layout(\n    title="Utilization of Operator 2 for 1000 Customers",\n    xaxis_title="Random Seeds",\n    yaxis_title="Utilization Percentage",\n    font=dict(\n        family="Courier New, monospace",\n        size=18,\n        color="#7f7f7f"\n    )\n)\nfig.show()'

In [23]:
"""fig = go.Figure([go.Bar(x = random_seeds , y = data5k["Operator2_Util"] )])
fig.update_layout(
    title="Utilization of Operator 2 for 5000 Customers",
    xaxis_title="Random Seeds",
    yaxis_title="Utilization Percentage",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"
    )
)
fig.show()"""

'fig = go.Figure([go.Bar(x = random_seeds , y = data5k["Operator2_Util"] )])\nfig.update_layout(\n    title="Utilization of Operator 2 for 5000 Customers",\n    xaxis_title="Random Seeds",\n    yaxis_title="Utilization Percentage",\n    font=dict(\n        family="Courier New, monospace",\n        size=18,\n        color="#7f7f7f"\n    )\n)\nfig.show()'

In [24]:
"""fig = go.Figure([go.Bar(x = random_seeds , y = data1k["Avg_queue_len_1"] )])
fig.update_layout(
    title="Average Length of Queue 1 for 1000 Customers",
    xaxis_title="Random Seeds",
    yaxis_title="Average Number of Ppl",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"
    )
)
fig.show()"""

'fig = go.Figure([go.Bar(x = random_seeds , y = data1k["Avg_queue_len_1"] )])\nfig.update_layout(\n    title="Average Length of Queue 1 for 1000 Customers",\n    xaxis_title="Random Seeds",\n    yaxis_title="Average Number of Ppl",\n    font=dict(\n        family="Courier New, monospace",\n        size=18,\n        color="#7f7f7f"\n    )\n)\nfig.show()'

In [25]:
"""fig = go.Figure([go.Bar(x = random_seeds , y = data1k["Avg_queue_len_2"] )])
fig.update_layout(
    title="Average Length of Queue 2 for 1000 Customers",
    xaxis_title="Random Seeds",
    yaxis_title="Average Number of Ppl",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"
    )
)
fig.show()"""

'fig = go.Figure([go.Bar(x = random_seeds , y = data1k["Avg_queue_len_2"] )])\nfig.update_layout(\n    title="Average Length of Queue 2 for 1000 Customers",\n    xaxis_title="Random Seeds",\n    yaxis_title="Average Number of Ppl",\n    font=dict(\n        family="Courier New, monospace",\n        size=18,\n        color="#7f7f7f"\n    )\n)\nfig.show()'

In [26]:
"""fig = go.Figure([go.Bar(x = random_seeds , y = data5k["Avg_queue_len_1"] )])
fig.update_layout(
    title="Average Length of Queue 1 for 5000 Customers",
    xaxis_title="Random Seeds",
    yaxis_title="Average Number of Ppl",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"
    )
)
fig.show()"""

'fig = go.Figure([go.Bar(x = random_seeds , y = data5k["Avg_queue_len_1"] )])\nfig.update_layout(\n    title="Average Length of Queue 1 for 5000 Customers",\n    xaxis_title="Random Seeds",\n    yaxis_title="Average Number of Ppl",\n    font=dict(\n        family="Courier New, monospace",\n        size=18,\n        color="#7f7f7f"\n    )\n)\nfig.show()'

In [27]:
"""fig = go.Figure([go.Bar(x = random_seeds , y = data5k["Avg_queue_len_2"] )])
fig.update_layout(
    title="Average Length of Queue 2 for 5000 Customers",
    xaxis_title="Random Seeds",
    yaxis_title="Average Number of Ppl",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"
    )
)
fig.show()"""

'fig = go.Figure([go.Bar(x = random_seeds , y = data5k["Avg_queue_len_2"] )])\nfig.update_layout(\n    title="Average Length of Queue 2 for 5000 Customers",\n    xaxis_title="Random Seeds",\n    yaxis_title="Average Number of Ppl",\n    font=dict(\n        family="Courier New, monospace",\n        size=18,\n        color="#7f7f7f"\n    )\n)\nfig.show()'

In [28]:
"""fig = go.Figure([go.Bar(x = random_seeds , y = data1k["MTWT/MTST"] )])
fig.update_layout(
    title="Maximum(Cumulative) Total Waiting Time to Total System Time Ratio for 1000 Customers",
    xaxis_title="Random Seeds",
    yaxis_title="The % Percentage of Ratio",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"
    )
)
fig.show()"""

'fig = go.Figure([go.Bar(x = random_seeds , y = data1k["MTWT/MTST"] )])\nfig.update_layout(\n    title="Maximum(Cumulative) Total Waiting Time to Total System Time Ratio for 1000 Customers",\n    xaxis_title="Random Seeds",\n    yaxis_title="The % Percentage of Ratio",\n    font=dict(\n        family="Courier New, monospace",\n        size=18,\n        color="#7f7f7f"\n    )\n)\nfig.show()'

In [29]:
"""fig = go.Figure([go.Bar(x = random_seeds , y = data5k["MTWT/MTST"] )])
fig.update_layout(
    title="Maximum(Cumulative) Total Waiting Time to Total System Time Ratio for 5000 Customers",
    xaxis_title="Random Seeds",
    yaxis_title="The % Percentage of Ratio",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"
    )
)
fig.show()"""

'fig = go.Figure([go.Bar(x = random_seeds , y = data5k["MTWT/MTST"] )])\nfig.update_layout(\n    title="Maximum(Cumulative) Total Waiting Time to Total System Time Ratio for 5000 Customers",\n    xaxis_title="Random Seeds",\n    yaxis_title="The % Percentage of Ratio",\n    font=dict(\n        family="Courier New, monospace",\n        size=18,\n        color="#7f7f7f"\n    )\n)\nfig.show()'

In [30]:
"""fig = go.Figure([go.Bar(x = random_seeds , y = data1k["Num_Of_Unsatisfied_Customers"] )])
fig.update_layout(
    title="Average Number of Customers Leaving The System Unsatisfied  for 1000 Customers",
    xaxis_title="Random Seeds",
    yaxis_title="The Avg. Number of Ppl",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"
    )
)
fig.show()"""

'fig = go.Figure([go.Bar(x = random_seeds , y = data1k["Num_Of_Unsatisfied_Customers"] )])\nfig.update_layout(\n    title="Average Number of Customers Leaving The System Unsatisfied  for 1000 Customers",\n    xaxis_title="Random Seeds",\n    yaxis_title="The Avg. Number of Ppl",\n    font=dict(\n        family="Courier New, monospace",\n        size=18,\n        color="#7f7f7f"\n    )\n)\nfig.show()'

In [31]:
"""fig = go.Figure([go.Bar(x = random_seeds , y = data5k["Num_Of_Unsatisfied_Customers"] )])
fig.update_layout(
    title="Average Number of Customers Leaving The System Unsatisfied  for 5000 Customers",
    xaxis_title="Random Seeds",
    yaxis_title="The Avg. Number of Ppl",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"
    )
)
fig.show()"""

'fig = go.Figure([go.Bar(x = random_seeds , y = data5k["Num_Of_Unsatisfied_Customers"] )])\nfig.update_layout(\n    title="Average Number of Customers Leaving The System Unsatisfied  for 5000 Customers",\n    xaxis_title="Random Seeds",\n    yaxis_title="The Avg. Number of Ppl",\n    font=dict(\n        family="Courier New, monospace",\n        size=18,\n        color="#7f7f7f"\n    )\n)\nfig.show()'

In [32]:
"""fig = go.Figure([go.Bar(x = random_seeds , y = data1k["avg_wait_time"] )])
fig.update_layout(
    title="Average Total Waiting Time for 1000 Customers",
    xaxis_title="Random Seeds",
    yaxis_title="Time in Minutes",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"
    )
)
fig.show()"""

'fig = go.Figure([go.Bar(x = random_seeds , y = data1k["avg_wait_time"] )])\nfig.update_layout(\n    title="Average Total Waiting Time for 1000 Customers",\n    xaxis_title="Random Seeds",\n    yaxis_title="Time in Minutes",\n    font=dict(\n        family="Courier New, monospace",\n        size=18,\n        color="#7f7f7f"\n    )\n)\nfig.show()'

In [33]:
"""fig = go.Figure([go.Bar(x = random_seeds , y = data5k["avg_wait_time"] )])
fig.update_layout(
    title="Average Total Waiting Time for 5000 Customers",
    xaxis_title="Random Seeds",
    yaxis_title="Time in Minutes",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"
    )
)
fig.show()"""

'fig = go.Figure([go.Bar(x = random_seeds , y = data5k["avg_wait_time"] )])\nfig.update_layout(\n    title="Average Total Waiting Time for 5000 Customers",\n    xaxis_title="Random Seeds",\n    yaxis_title="Time in Minutes",\n    font=dict(\n        family="Courier New, monospace",\n        size=18,\n        color="#7f7f7f"\n    )\n)\nfig.show()'